## compare the coverage of three pangenome to *S.cerevisiae* genome 
- Through s228c&CEN.PK BLASTp to the 3 pan-genome to evaluate the coverage to sce genes of the 3 pan-genome
- blastp pan1800_v2 vs lgpangenome/napangenome



In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
os.chdir(r'D:\code\github/Unified_Yeast_GEMs_Database')
import sys
sys.path.append("code")
from mainFunction import get_gene_lens

In [3]:
def parse_blastp_result(blastp_file,blastp_dir,query,query_dir,subject,subject_dir):
    # load blastp result file
    df_blastp_file = pd.read_csv(blastp_dir+blastp_file, sep="\t", header=None, index_col=0)
    columns = ["subject", "identity", "alignment length", "mismatches", "gap opens", "q_start", "q_end", "s_start",
               "s_end", "evalue", "bit score"]
    df_blastp_file.columns = columns

    # get gene length for query and subject
    query_lens=get_gene_lens(query,in_folder=query_dir)
    subject_lens=get_gene_lens(subject,in_folder=subject_dir)
    query_lens.set_index("gene",inplace=True)
    subject_lens.set_index("gene",inplace=True)

    # map query lens to blastp_file and name the column as "query_lens"
    df_blastp_file=df_blastp_file.join(query_lens,how="left")
    df_blastp_file.rename(columns={"gene_length":"query_lens"},inplace=True)
    # map subject lens to blastp_file and name the column as "subject_lens"
    df_blastp_file=df_blastp_file.join(subject_lens,how="left",on="subject")
    df_blastp_file.rename(columns={"gene_length":"subject_lens"},inplace=True)

    # calculate COV for query and subject
    df_blastp_file["query_cov"]=(df_blastp_file["q_end"]-df_blastp_file["q_start"]+1)/df_blastp_file["query_lens"]
    df_blastp_file["subject_cov"]=(df_blastp_file["s_end"]-df_blastp_file["s_start"]+1)/df_blastp_file["subject_lens"]

    return df_blastp_file


def blastp_result_filter(df_blastp_result,cov,pid):
    df_filtered=df_blastp_result[(df_blastp_result['identity']>=pid)&(df_blastp_result["query_cov"]>=cov)&(df_blastp_result["subject_cov"]>=cov)]
    return df_filtered

In [16]:
query_dir=r"data\genome"
query="S288c_R64.fasta"
blastp_dir=r"code/3.pan-genome_construction/3.pan-genome_comparison/output/"
cenpk_genenumb=5451
s288c_genenumb=6717
# parse blastp result
df_napan_blastp=parse_blastp_result(blastp_dir=blastp_dir,
                                    blastp_file="s288c_vs_pan1011_blastp_result.txt",
                                    query=query,
                                    query_dir=query_dir,
                                    subject="na_pan1011.fasta",
                                    subject_dir=r"data\genome")

df_lgpan_blastp=parse_blastp_result(blastp_dir=blastp_dir,
                                    blastp_file="s288c_vs_lgpan_blastp_result.txt",
                                    query=query,
                                    query_dir=query_dir,
                                    subject="lg_pan1392.fasta",
                                    subject_dir=r"data\genome")

df_pan1800_blastp=parse_blastp_result(blastp_dir=blastp_dir,
                                      blastp_file="s288c_vs_pan1800_50_70_v2.txt",
                                      query=query,
                                      query_dir=query_dir,
                                      subject="pan1800.fasta",
                                      subject_dir=r"data\genome")


# filte blastp result
pid=0.7
cov=0.5
df_lgpan_blastp=blastp_result_filter(df_lgpan_blastp,cov,pid)
df_napan_blastp=blastp_result_filter(df_napan_blastp,cov,pid)
df_pan1800_blastp=blastp_result_filter(df_pan1800_blastp,cov,pid)


In [17]:
df_mapping_gene_count=pd.DataFrame(index=["mapped s288c number","hit pan-genome number"])
df_mapping_gene_count["na1011 pangenome"]=[len(set(df_napan_blastp.index.tolist())),len(set(df_napan_blastp["subject"].tolist()))]
df_mapping_gene_count["lg1392 pangenome"]=[len(set(df_lgpan_blastp.index.tolist())),len(set(df_lgpan_blastp["subject"].tolist()))]
df_mapping_gene_count["this1800 pangenome"]=[len(set(df_pan1800_blastp.index.tolist())),len(set(df_pan1800_blastp["subject"].tolist()))]
df_mapping_gene_count.loc['s288c gene number',:]=[s288c_genenumb] * 3
df_mapping_gene_count.T

,mapped s288c number,hit pan-genome number,s288c gene number
na1011 pangenome,6051.0,6850.0,6717.0
lg1392 pangenome,6231.0,5968.0,6717.0
this1800 pangenome,6688.0,6276.0,6717.0


### CEN.PK BLASTp to the 3 pan-genome

In [6]:
query_dir=r"data\genome"
query="cenpk1.fasta"
blastp_dir=r"code/3.pan-genome_construction/3.pan-genome_comparison/output/"

df_napan_cenpk_blastp=parse_blastp_result(blastp_dir=blastp_dir,
                                    blastp_file="cenpk_vs_pan1011_blastp.txt",
                                    query=query,
                                    query_dir=query_dir,
                                    subject="na_pan1011.fasta",
                                    subject_dir=r"data\genome")

df_lgpan_cenpk_blastp=parse_blastp_result(blastp_dir=blastp_dir,
                                    blastp_file="cenpk_vs_lgpan_blastp.txt",
                                    query=query,
                                    query_dir=query_dir,
                                    subject="lg_pan1392.fasta",
                                    subject_dir=r"data\genome")

df_pan1800_cenpk_blastp=parse_blastp_result(blastp_dir=blastp_dir,
                                      blastp_file="cenpk_vs_pan1800_50_70_v2.txt",
                                      query=query,
                                      query_dir=query_dir,
                                      subject="pan1800.fasta",
                                      subject_dir=r"data\genome")



df_lgpan_cenpk_blastp=blastp_result_filter(df_lgpan_cenpk_blastp,cov=0.5,pid=0.7)
df_napan_cenpk_blastp=blastp_result_filter(df_napan_cenpk_blastp,cov=0.5,pid=0.7)
df_pan1800_cenpk_blastp=blastp_result_filter(df_pan1800_cenpk_blastp,cov=0.5,pid=0.7)


In [6]:
df_mapping_cenpk_gene_count=pd.DataFrame(index=["mapped CEN.pk number","hit pan-genome number"])
df_mapping_cenpk_gene_count["na1011 pangenome"]=[len(set(df_napan_cenpk_blastp.index.tolist())),len(set(df_napan_cenpk_blastp["subject"].tolist()))]
df_mapping_cenpk_gene_count["lg1392 pangenome"]=[len(set(df_lgpan_cenpk_blastp.index.tolist())),len(set(df_lgpan_cenpk_blastp["subject"].tolist()))]
df_mapping_cenpk_gene_count["this1800 pangenome"]=[len(set(df_pan1800_cenpk_blastp.index.tolist())),len(set(df_pan1800_cenpk_blastp["subject"].tolist()))]
df_mapping_cenpk_gene_count.T

,mapped CEN.pk number,hit pan-genome number
na1011 pangenome,5202,6075
lg1392 pangenome,4989,4906
this1800 pangenome,5351,5218


In [11]:
# df_pan1800_cenpk_blastp
# df_pan1800_blastp
# len(set(list(df_pan1800_cenpk_blastp["subject"].unique())+list(df_pan1800_blastp["subject"].unique())))


- conclusion:<br>
    CEN.PK and S288c BLASTp show the same result that new pan-genome has a better coverage to S.cerevisiae strain proteome

### parse blastp result:pan1800_v2 vs lgpangenome/napangenome

In [14]:
blastp_result_dir=r"code\3.pan-genome_construction\3.pan-genome_comparison\output/"
df_pan1800_napan_blastp=parse_blastp_result(blastp_dir=blastp_result_dir,
                                            blastp_file="pan1800_v2_vs_pan1011_blastp.txt",
                                            query="pan1800_50_70_v2.fasta",
                                            query_dir=r"data\genome",
                                            subject="pan1011_v1.fasta",
                                            subject_dir=r"data\genome")
df_pan1800_lgpan_blastp=parse_blastp_result(blastp_dir=blastp_result_dir,
                                            blastp_file="pan1800_v2_vs_lgpan_blastp.txt",
                                            query="pan1800_50_70_v2.fasta",
                                            query_dir=r"data\genome",
                                            subject="lg_pangenome.fasta",
                                            subject_dir=r"data\genome")

# df_pan1800_napan_blastp=blastp_result_filter(df_pan1800_napan_blastp,cov=0.4,pid=0.5)
# df_pan1800_lgpan_blastp=blastp_result_filter(df_pan1800_lgpan_blastp,cov=0.4,pid=0.5)


In [15]:
df_mapping_pan1800_gene_count=pd.DataFrame(index=["mapped pan1800 number","hit pan-genome number"])
df_mapping_pan1800_gene_count["na1011 pangenome"]=[len(set(df_pan1800_napan_blastp.index.tolist())),len(set(df_pan1800_napan_blastp["subject"].tolist()))]
df_mapping_pan1800_gene_count["lg1392 pangenome"]=[len(set(df_pan1800_lgpan_blastp.index.tolist())),len(set(df_pan1800_lgpan_blastp["subject"].tolist()))]
df_mapping_pan1800_gene_count.T

,mapped pan1800 number,hit pan-genome number
na1011 pangenome,6612,7473
lg1392 pangenome,7007,6583


- conclusion:<br>
    according to pan1800_vs_napangenome blastp result, 7473 genes only hit to 6612 pan1800 genes,indicating that na1011 pangenome may be redundant